In [1]:
import warnings
warnings.filterwarnings("ignore")
from statsmodels.tsa.api import ExponentialSmoothing,SimpleExpSmoothing,Holt
#Working on plotly
import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
cf.go_offline()
import pandas as pd
from scipy.stats import kurtosis
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [2]:
# Import the file
df=pd.read_csv(r"C:\Users\2000139\TORQ_avg_1.txt")
df.head()

,Number of points: 1029419
0,Sampling rate(Hz): 0.0166666666666667
1,Time range: From: 05/10/2016 08:44:54; Until: ...
2,Units: kNm
3,01.06.2016 00:00:13\t28.9
4,01.06.2016 00:01:13\t29


In [3]:
#Preprocessing steps
col_name='TORQ_avg_1'
df.columns=[col_name]
df=pd.DataFrame(df[col_name].str.split('\t',2).tolist())
df.dropna(inplace=True)
df[0]=pd.to_datetime(df[0],format='%d.%m.%Y %H:%M:%S')
df.index=df[0]
df.index=df.index.date
df.drop(columns=[0],inplace=True)
df.columns=[col_name]
df.isnull().sum()
df=df.astype({col_name:float,})
df.info()
df=df[382246:425509]
df

<class 'pandas.core.frame.DataFrame'>
Index: 1029419 entries, 2016-06-01 to 2018-06-30
Data columns (total 1 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   TORQ_avg_1  1029419 non-null  float64
dtypes: float64(1)
memory usage: 15.7+ MB


,TORQ_avg_1
2017-03-01,1.4
2017-03-01,1.4
2017-03-01,1.4
2017-03-01,1.4
2017-03-01,1.4
...,...
2017-03-31,0.0
2017-03-31,0.0
2017-03-31,0.0
2017-03-31,0.0


In [4]:
def stastical_data(df,col_name):
    final_df=df.groupby(df.index).agg({col_name:['min','max','mean','std']})
    header = np.array(['Min','Max','Avg', 'Standard_deviation'])
    final_df.columns=header
    print("Pre-Processing is done")
    fig=go.Figure()
    fig.add_trace(go.Scatter(x=final_df.index,y=final_df['Max'],mode='lines',name=col_name+' (Max)'))
    fig.add_trace(go.Scatter(x=final_df.index,y=final_df['Min'],mode='lines',name=col_name+' (Min)'))
    fig.add_trace(go.Scatter(x=final_df.index,y=final_df['Avg'],mode='lines',name=col_name+' (Avg)'))
    fig.add_trace(go.Scatter(x=final_df.index,y=final_df['Standard_deviation'],mode='lines',name=col_name+' (Standard_deviation)'))
    fig.update_layout(hovermode="x unified")
    fig.update_layout(
        title=col_name,
        xaxis_title="Date",
        yaxis_title="Values",
        legend_title="Statistical Parameters")
    fig.write_html("TORQ\Plots\\"+col_name+"_Stats.html")
    final_df.to_csv("TORQ\Data\\"+col_name+"_Stats.csv") 
    print("completed")

In [5]:
stastical_data(df,col_name)

Pre-Processing is done
completed


In [6]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=df.index,y=df[col_name],mode='lines',name=col_name+'(Waveform)'))
fig.update_layout(hovermode="x unified")
fig.update_layout(
    title=col_name,
    xaxis_title="Date",
    yaxis_title="Values",
    legend_title="Statistical Parameters")
fig.write_html("TORQ\Plots\\"+col_name+"_Waveform.html")